In [51]:
import torch
import cv2
import tqdm
import matplotlib as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [52]:
meta_file = '../data/new_augmented.csv'
meta = pd.read_csv(meta_file)
meta.drop(['Unnamed: 0'],axis=1,inplace=True)
meta.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,../data/Augmented_rename/ISIC_0031633.jpg


In [54]:
meta['new_path'] = '../data/combined_data/'+meta['image_id']+'.jpg'

In [55]:
meta.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path,new_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0027419.jpg,../data/combined_data/ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0025030.jpg,../data/combined_data/ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0026769.jpg,../data/combined_data/ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0025661.jpg,../data/combined_data/ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,../data/Augmented_rename/ISIC_0031633.jpg,../data/combined_data/ISIC_0031633.jpg


In [ ]:
meta.drop(['Unnamed: 0'],axis=1,inplace=True)

In [56]:
X, y = meta.iloc[:,[0,1,3,4,5,6,7,8]], meta.iloc[:,2]

In [57]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, stratify=y,random_state=42)
print(f"train: {len(xtrain)}, test: {len(xtest)} ")

train: 37596, test: 9400 


In [ ]:
ytrain.value_counts()

In [ ]:
ytest.value_counts()

In [58]:
xtrain.insert(loc = 2, column = 'dx', value = ytrain)


In [59]:
xtest.insert(loc= 2 , column = 'dx', value = ytest )

In [60]:
xtest.to_csv('../data/meta_augment_test.csv', index=False)
xtrain.to_csv('../data/meta_augment_train.csv', index=False)

In [8]:
xtrain

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
33488,HAM_0007097,ISIC_0033230_28,vasc,histo,60.0,male,trunk,../data/Augmented_rename/ISIC_0033230_28.jpg
25236,HAM_0004522,ISIC_0028381_8,akiec,histo,55.0,female,neck,../data/Augmented_rename/ISIC_0028381_8.jpg
29020,HAM_0005718,ISIC_0029549_12,akiec,histo,70.0,male,back,../data/Augmented_rename/ISIC_0029549_12.jpg
1121,HAM_0007158,ISIC_0027206,nv,follow_up,50.0,female,lower extremity,../data/HAM10k/HAM10000_images/ISIC_0027206.jpg
36044,HAM_0007532,ISIC_0024662_46,vasc,consensus,75.0,male,trunk,../data/Augmented_rename/ISIC_0024662_46.jpg
...,...,...,...,...,...,...,...,...
20716,HAM_0001749,ISIC_0033372_12,bcc,histo,70.0,male,chest,../data/Augmented_rename/ISIC_0033372_12.jpg
27499,HAM_0006483,ISIC_0032371_11,akiec,histo,75.0,male,upper extremity,../data/Augmented_rename/ISIC_0032371_11.jpg
33856,HAM_0005471,ISIC_0028146_20,vasc,histo,60.0,female,back,../data/Augmented_rename/ISIC_0028146_20.jpg
11225,HAM_0000337,ISIC_0026081_4,mel,histo,50.0,female,back,../data/Augmented_rename/ISIC_0026081_4.jpg


In [12]:
xtrain[xtrain['dx']=='nv']

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
1121,HAM_0007158,ISIC_0027206,nv,follow_up,50.0,female,lower extremity,../data/HAM10k/HAM10000_images/ISIC_0027206.jpg
4366,HAM_0006391,ISIC_0032783,nv,histo,70.0,female,lower extremity,../data/HAM10k/HAM10000_images/ISIC_0032783.jpg
5810,HAM_0001014,ISIC_0031278,nv,histo,70.0,male,back,../data/HAM10k/HAM10000_images/ISIC_0031278.jpg
6446,HAM_0001017,ISIC_0024507,nv,consensus,30.0,female,abdomen,../data/HAM10k/HAM10000_images/ISIC_0024507.jpg
4443,HAM_0006539,ISIC_0033798,nv,histo,40.0,male,back,../data/HAM10k/HAM10000_images/ISIC_0033798.jpg
...,...,...,...,...,...,...,...,...
649,HAM_0004070,ISIC_0027657,nv,follow_up,40.0,male,trunk,../data/HAM10k/HAM10000_images/ISIC_0027657.jpg
4008,HAM_0002575,ISIC_0026781,nv,histo,55.0,female,back,../data/HAM10k/HAM10000_images/ISIC_0026781.jpg
3898,HAM_0001316,ISIC_0031660,nv,histo,45.0,male,back,../data/HAM10k/HAM10000_images/ISIC_0031660.jpg
1507,HAM_0001253,ISIC_0025704,nv,follow_up,55.0,female,back,../data/HAM10k/HAM10000_images/ISIC_0025704.jpg


In [13]:
import glob

In [17]:
print(len(glob.glob('../data/Augmented_rename/*.jpg')) + len(glob.glob('../data/HAM10k/HAM10000_images/*.jpg')))

47146


In [20]:
47146-46996

150

In [24]:
aug_r = glob.glob(r'../data/Augmented_rename/*.jpg')
orig = glob.glob(r'../data/HAM10k/HAM10000_images/*.jpg')

In [27]:
meta

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,../data/Augmented_rename/ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,../data/Augmented_rename/ISIC_0031633.jpg
...,...,...,...,...,...,...,...,...
46991,HAM_0002153,ISIC_0029297_53,df,consensus,40.0,female,lower extremity,../data/Augmented_rename/ISIC_0029297_53.jpg
46992,HAM_0002153,ISIC_0029297_54,df,consensus,40.0,female,lower extremity,../data/Augmented_rename/ISIC_0029297_54.jpg
46993,HAM_0002153,ISIC_0029297_55,df,consensus,40.0,female,lower extremity,../data/Augmented_rename/ISIC_0029297_55.jpg
46994,HAM_0002153,ISIC_0029297_56,df,consensus,40.0,female,lower extremity,../data/Augmented_rename/ISIC_0029297_56.jpg


In [47]:
original

0

In [48]:
aug

10015